Wiki Scraper
---

Author: Peter Zhang

Scraping tool for the Wiki.

### Setup

#### Imports

In [1]:
# imports
import urllib.request, urllib.parse, urllib.error
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import csv
import os.path
from os import path
import re

#### Settings

- OVERWRITE determines whether or not to update existing files.
- PAGES_URL is a list of Wiki pages
- OUTPATH is where files are stored

In [9]:
# settings
OVERWRITE = True
TARGET_WIKIS = ["PF17", "PF18", "PF19", "PF20"]

In [10]:
# get page URLs
WIKIS_URL = "tools/wiki_pages.csv"
WIKIS = [row for row in csv.reader(open(WIKIS_URL, 'r'))]

In [11]:
# outpath
OUTPATH = "wiki_data/"

### Scrapers

#### Wiki Page

Take the URL to a wiki page and returns a list of all the school names and their respective URLs.

In [12]:
# for a given archive year, return all schools
def getSchools(url):
    
    # try opening URL
    try:
        html = urlopen(url).read()
    except Exeption:
        print("Wiki URL broke")
        return []
    
    
    # soupify
    soup = BeautifulSoup(html, "html.parser")
    
    # find all links
    links = soup.find_all('a')
    
    # find only school links
    links = [link for link in links if "(" in link.text and ")" in link.text]
    
    # get link text and URL
    schools = [[link.text, link.get('href')] for link in links]

    return schools

In [13]:
getSchools("https://hspolicy19.debatecoaches.org/")

[['Advanced Technologies Academy (NV)', '/Advanced%20Technologies%20Academy/'],
 ['Airline (AL)', '/Airline/'],
 ['Aledo (TX)', '/Aledo/'],
 ['Alief Taylor (TX)', '/Alief%20Taylor/'],
 ['Alpharetta (GA)', '/Alpharetta/'],
 ['Altamont (AL)', '/Altamont/'],
 ['American Heritage (FL)', '/American%20Heritage/'],
 ['Anderson (TX)', '/Anderson/'],
 ['Andover (KS)', '/Andover/'],
 ['Andover Central (KS)', '/Andover%20Central/'],
 ['Andrews (TX)', '/Andrews/'],
 ['Archbishop Mitty (CA)', '/Archbishop%20Mitty/'],
 ['Ashland (OR)', '/Ashland/'],
 ['Asian Debate League (AK)', '/Asian%20Debate%20League/'],
 ['Athens (TX)', '/Athens/'],
 ['Aubrey (TX)', '/Aubrey/'],
 ['BASIS Chandler (AZ)', '/BASIS%20Chandler/'],
 ['BASIS Peoria (AZ)', '/BASIS%20Peoria/'],
 ['BASIS Shavano (TX)', '/BASIS%20Shavano/'],
 ['Bakersfield (CA)', '/Bakersfield/'],
 ['Baltimore City College (MD)', '/Baltimore%20City%20College/'],
 ['Barstow (MO)', '/Barstow/'],
 ['Baton Rouge Magnet (LA)', '/Baton%20Rouge%20Magnet/'],
 ['B

#### School Page

Take the URL to a school and return a list of debaters and the URLs that point to their Aff and Neg pages

In [14]:
# return aff and neg URLs for all the debaters of a given school

def getTeams(url):
    
    # try opening URL
    try:
        html = urlopen(url).read()
    except Exception:
        return (0, [], {}, "School URL broke")
        
    # soupify
    soup = BeautifulSoup(html, "html.parser")
    
    # look for a table
    if len(soup.find_all("table")):
        table = soup.find_all("table")[0]
    else:
        return (0, [], {}, "School page not set up")
        
    # exclude header
    numDebaters = len(table.find_all('tr'))-1
    
    if numDebaters == 0:
        return (0, [], {}, "No debaters listed")
    
    # track team names and map to URLs
    teamNames = []
    pageURLs = {}
    
    
    # for each table row
    for row in table.find_all('tr')[1:]:
        
        # the first column's string is the team name
        team = row.find('td').string
        teamNames.append(team)
        
        # collect the URLs to the aff and neg page
        links = row.find_all('a')
        URLs = [link.get("href") for link in links]
        pageURLs[team] = URLs
    
    return (numDebaters, teamNames, pageURLs, "")


In [15]:
getTeams("https://hspolicy19.debatecoaches.org/Advanced%20Technologies%20Academy")

(3,
 ['Advanced Technologies Academy Jacqueline Balanovsky - Raymond Behnke',
  'Advanced Technologies Academy Sam Self - Jonah Gentleman',
  'Advanced Technologies Academy Ryan Fritchel - Hannah Lewis'],
 {'Advanced Technologies Academy Jacqueline Balanovsky - Raymond Behnke': ['/Advanced%20Technologies%20Academy/Balanovsky-Behnke%20Aff',
   '/Advanced%20Technologies%20Academy/Balanovsky-Behnke%20Neg'],
  'Advanced Technologies Academy Sam Self - Jonah Gentleman': ['/Advanced%20Technologies%20Academy/Self-Gentleman%20Aff',
   '/Advanced%20Technologies%20Academy/Self-Gentleman%20Neg'],
  'Advanced Technologies Academy Ryan Fritchel - Hannah Lewis': ['/Advanced%20Technologies%20Academy/Fritchel-Lewis%20Aff',
   '/Advanced%20Technologies%20Academy/Fritchel-Lewis%20Neg']},
 '')

#### Team Page

In [16]:
def getDisclosure(url):

    # try opening URL
    try:
        html = urlopen(url).read().decode('utf-8-sig')
        
    except Exception:
        return {"Team URL":url,
                "Rounds": [],
                "Round Reports": [],
                "Cites": [],
                "Errors": "Did not open"}
    
    
    # soupify
    soup = BeautifulSoup(html, "html.parser")
    
    tables = soup.find_all("table")

    if len(tables) > 2:
        
        rnds = tables[0]
        rndrprts = tables[1]
        cts = tables[2]
    
    else:
        
        return {"Team URL":url,
                "Rounds": [],
                "Round Reports": [],
                "Cites": [],
                "Errors": "Not configured properly"}
    
    # collect all rounds
    rounds = []
    
    for row in rnds.find_all("tr")[1:]:
        
        # get pieces
        cols = row.find_all("td")
        trn = cols[0].text
        rnd = cols[1].text
        opp = cols[2].text
        jdg = cols[3].text
        osrc = cols[6].find("a")
        if osrc:
            docURL = osrc.get("href")
        else:
            docURL = ""
            
        rounds.append([trn, rnd, opp, jdg, docURL])
    
    # collect all round reports
    roundReports = []
    
    for row in rndrprts.find_all("tr")[1:]:
        try:
            pars = roundReports.append(row.find_all("td")[2].find_all("p")[1].text)
        except:
            continue

    # collect all cites
    cites = [row.find_all("td")[0].find("span").text for row in cts.find_all('tr')[1:]]
        
    return {"Team URL":url,
            "Rounds": rounds,
            "Round Reports": roundReports,
            "Cites": cites,
            "Errors": ""}
    


In [17]:
getDisclosure("https://hsld17.debatecoaches.org/Acton-Boxborough/Liu%20Aff")

{'Team URL': 'https://hsld17.debatecoaches.org/Acton-Boxborough/Liu%20Aff',
 'Rounds': [['-', 'Finals', '-', '-', ''],
  ['Harvard', '2', 'Dan Shahab', 'David Moon', ''],
  ['Lexington Winter Invitational', '2', 'Scarsdale AW', 'David Moon', ''],
  ['Newark Invitational', '4', 'Stuyvesant JL', 'Amit Kukreja', ''],
  ['Practice Round', '1', 'Michelle I drop T Li', 'the coolest judge', ''],
  ['Practice Round', 'Semis', '-', '-', '']],
 'Round Reports': [],
 'Cites': ['Contact',
  'JF - Brown Bear AC',
  'JF - Brown Bear AC v2',
  'JF - Brown Bear AC v3',
  'JF - Deleuzian Narrativity 1AC'],
 'Errors': ''}

#### Execution

Loop through wikis, find all schools, visit all entry pages, scrape all positions.

In [ ]:
for wiki in WIKIS:
    
    # unpack wiki
    wikiName = wiki[0]
    wikiURL = wiki[1]
    
    # if not in target wikis, don't write
    if wikiName not in TARGET_WIKIS:
        continue
    
    # if it exists, don't overwrite
    if (not OVERWRITE) and path.exists(OUTPATH + wikiName + "_schools_wiki.csv"):
        continue
        
    # open output file
    with open(OUTPATH + wikiName + "schools_wiki.csv", 'w', encoding="utf-8") as SCHOOLFILE, open(OUTPATH + wikiName + "teams_wiki.csv", 'w', encoding="utf-8") as TEAMFILE:
        
        # instantaite writer
        schoolWriter = csv.DictWriter(SCHOOLFILE,
                                     fieldnames = ["School Name",
                                                   "School URL",
                                                   "School Teams",
                                                   "Errors"],
                                      quotechar='"', 
                                      quoting=csv.QUOTE_NONNUMERIC,
                                     lineterminator = "\n")
        
        # write header
        schoolWriter.writeheader()
        
        # instantaite writer
        teamWriter = csv.DictWriter(TEAMFILE,
                                    fieldnames = ["Team Name",
                                                  "Side",
                                                  "Team URL",
                                                  "Rounds",
                                                  "Round Reports",
                                                  "Cites",
                                                  "Errors"],
                                    lineterminator = "\n")
        
        # write header
        teamWriter.writeheader()
            
        print("Scraping " + wikiName)
        
        # get school URLs
        schools = getSchools(wikiURL)
        
        print("There are " + str(len(schools)) + " schools")
        
        for school in schools:
            
            # get school name and URL
            schoolName, schoolURL = school[0], wikiURL + school[1]
            
            print("Checking " + schoolName)
            
            # get teams from the school
            numTeams, teamNames, teamURLs, errors = getTeams(schoolURL)
            
            print("Found " + str(numTeams) + " teams")
            
            # write school information
            schoolWriter.writerow({"School Name" : schoolName,
                                  "School URL": schoolURL,
                                  "School Teams": teamNames,
                                  "Errors": errors})
            
            for team in teamNames:
                
                try:
                
                    print("Scraping " + team)

                    # get team URL
                    teamURL = teamURLs[team]
                    affURL = wikiURL + teamURL[0]
                    negURL = wikiURL + teamURL[1]

                    # aff scraping
                    affInfo = getDisclosure(affURL)
                    affInfo["Team Name"] = team
                    affInfo["Side"] = "Aff"

                    # write aff info
                    teamWriter.writerow(affInfo)

                    # neg scraping
                    negInfo = getDisclosure(negURL)
                    negInfo["Team Name"] = team
                    negInfo["Side"] = "Neg"

                    # write neg info
                    teamWriter.writerow(negInfo)
                    
                except:
                    
                    print(team + " failed.")
                    
                

Scraping PF17
There are 597 schools
Checking AM Consolidated (TX)
Found 0 teams
Checking Abernathy (TX)
Found 0 teams
Checking Acton-Boxborough (MA)
Found 0 teams
Checking ACORN Community (NY)
Found 0 teams
Checking Agape Leaders Prep (NY)
Found 0 teams
Checking Airline (LA)
Found 0 teams
Checking Albuquerque Academy (NM)
Found 0 teams
Checking Alpharetta (GA)
Found 0 teams
Checking Alpine (UT)
Found 0 teams
Checking Alta (UT)
Found 0 teams
Checking Altamont (AL)
Found 0 teams
Checking Anderson (TX)
Found 2 teams
Scraping Anderson John David Erard - Callum Harralson
Scraping Anderson Mark Foster - Cody Builta
Checking Andrew (TX)
Found 0 teams
Checking Appleton East (WI)
Found 0 teams
Checking Appleton (MD)
Found 0 teams
Checking Arapahoe (CO)
Found 0 teams
Checking Arcadia (CA)
Found 0 teams
Checking Archbishop Mitty (CA)
Found 4 teams
Scraping Archbishop Mitty Rohit Ghoshal - Shriya Nandwani
Scraping Archbishop Mitty Jolie Leung - Teja Veeramacheneni
Scraping Archbishop Mitty Roshni 

Scraping Des Moines Roosevelt Finn Andersen - Logan Garland
Scraping Des Moines Roosevelt Sri Chilukuri - Isaac Appel
Checking Desert Vista (AZ)
Found 0 teams
Checking Detroit Country Day (MI)
Found 0 teams
Checking Dexter (MI)
Found 0 teams
Checking Diamond Bar (CA)
Found 0 teams
Checking Dominion (VA)
Found 0 teams
Checking Dougherty Valley (CA)
Found 4 teams
Scraping Dougherty Valley Saad Jamal - Shreyas Kiran
Scraping Dougherty Valley Srisai Nachuri - Gabe Lee
Scraping Dougherty Valley Riya Bindlish - Sanji Ranganathan
Scraping Dougherty Valley Tanaya Bhakat - Vivian Kuang
Checking Dow (MI)
Found 0 teams
Checking Dowling Catholic (IA)
Found 0 teams
Checking Downtown Magnets (CA)
Found 0 teams
Checking Dripping Springs (TX)
Found 0 teams
Checking Duchesne Academy (TX)
Found 2 teams
Scraping Duchesne Academy Danielle Linbeck - Avnika Dubey
Scraping Duchesne Academy Brooke Blanton - Sylvia Duarte
Checking Dulles (TX)
Found 0 teams
Checking Dunwoody (GA)
Found 0 teams
Checking Dupont M

Found 0 teams
Checking Leucadia Independent (CA)
Found 0 teams
Checking Lexington (MA)
Found 0 teams
Checking Liberal Arts & Science Academy (TX)
Found 0 teams
Checking Lincoln (OR)
Found 0 teams
Checking Lincoln College (KS)
Found 0 teams
Checking Lincoln College (MO)
Found 0 teams
Checking Lincoln East (NE)
Found 0 teams
Checking Lincoln HS (NE)
Found 0 teams
Checking Lincoln Park (IL)
Found 0 teams
Checking Lincoln Sudbury (MA)
Found 1 teams
Scraping Lincoln Sudbury Lincoln Sudbury - Team Cases
Checking Lindale (TX)
Found 0 teams
Checking Lindblom Math&Science (IL)
Found 0 teams
Checking Lionel Wilson (CA) 
Found 0 teams
Checking Little Cypress Mauriceville (TX)
Found 0 teams
Checking Little Rock Central (AR)
Found 0 teams
Checking Little Rock Hall (AR)
Found 0 teams
Checking Littleton Independent (CO)
Found 0 teams
Checking Logan (UT)
Found 0 teams
Checking Lone Peak (UT)
Found 0 teams
Checking Los Altos (CA)
Found 1 teams
Scraping Los Altos Franklin Ruan - Jiayan Luo
Checking Lowe

Found 0 teams
Checking Richardson (TX)
Found 0 teams
Checking Ridge Point (TX)
Found 0 teams
Checking River Hill (MD)
Found 0 teams
Checking Riverside STEM (CA)
Found 0 teams
Checking Riverwood (GA)
Found 0 teams
Checking Robert McQueen (NV)
Found 0 teams
Checking Rocky Mountain (CO)
Found 0 teams
Checking Rogers Heritage (AR)
Found 0 teams
Checking Ronald Reagan (WI)
Found 0 teams
Checking Rosemont (CA)
Found 0 teams
Checking Rosemount (MN)
Found 0 teams
Checking Roseville (MN)
Found 0 teams
Checking Roswell (GA)
Found 0 teams
Checking Round Rock (TX)
Found 0 teams
Checking Rowland Hall (UT)
Found 3 teams
Scraping Rowland Hall Soren Feola - Isaac Ball
Scraping Rowland Hall Charlotte Orford - Aislinn Mitcham
Scraping Rowland Hall Robin Zeng - Celia Davis
Checking Rowlett (TX)
Found 0 teams
Checking Roy (UT)
Found 0 teams
Checking Rufus King (WI)
Found 0 teams
Checking Ruston (LA)
Found 0 teams
Checking Sage Ridge (NV)
Found 0 teams
Checking Saginaw (TX)
Found 0 teams
Checking Saint Mar

Found 0 teams
Checking Archbishop Mitty (CA)
Found 0 teams
Checking Ardrey Kell (NC)
Found 0 teams
Checking Ashland (OR)
Found 0 teams
Checking Asian Debate League (INT)
Found 0 teams
Checking Athens (TX)
Found 1 teams
Scraping Athens Francisco Garcia - Oscar Alfaro
Checking Atholton (MD)
Found 0 teams
Checking Austin (TX)
Found 0 teams
Checking Austin SFA (TX)
Found 0 teams
Checking Bakersfield (CA)
Found 0 teams
Checking Balboa (CA)
Found 0 teams
Checking Ballard (WA)
Found 0 teams
Checking Baltimore City College (MD)
Found 0 teams
Checking Barbers Hill (TX)
Found 0 teams
Checking Barstow (MO)
Found 0 teams
Checking BASIS Chandler (AZ)
Found 0 teams
Checking BASIS DC (DC)
Found 0 teams
Checking BASIS Independent Silicon Valley (CA)
Found 2 teams
Scraping BASIS Independent Silicon Valley Stefan Boone - Daniel Chen
Scraping BASIS Independent Silicon Valley Ayush Agarwal - Daphne Lu
Checking BASIS Peoria (AZ)
Found 1 teams
Scraping BASIS Peoria Sukesh Ram - David Parau
Checking Beacon (

...

Scraping North Broward Cassandra Ameen - Kailey Brown
Scraping North Broward Kailey Brown - Grant Cooper
Scraping North Broward Tyler Katz - Sebastian Rabbini
Scraping North Broward Tyler Brown - Tyler Katz
Checking North Mecklenburg (NC)
Found 2 teams
Scraping North Mecklenburg Robert MurrayGramlich - Serena Raj
Scraping North Mecklenburg Bernie Shen - Robert MurrayGramlich
Checking Northridge (UT)
Found 1 teams
Scraping Northridge Annee Farner - Blaine Cantwell
Checking Northview (GA)
Found 2 teams
Scraping Northview Ian Yang - Sreyaash Das
Scraping Northview Adya Bhatta - Sonal Churiwal
Checking Novices Of Zhang Institute (TX)
Found 1 teams
Scraping Novices Of Zhang Institute Timmy Simp - Tommy Simp
Checking Nueva (CA)
Found 1 teams
Scraping Nueva Pranav Ram - Noah Tavares
Checking Oakton (VA)
Found 6 teams
Scraping Oakton Lawrence Zhu - Nathaniel Yoon
Scraping Oakton Zoe Spicer - Nathaniel Yoon
Scraping Oakton Lawrence Zhu - Isaac Ratzan
Scraping Oakton Chas Huang - Andrew Jones
Sc

Found 1 teams
Scraping Strakethesda Prep Ben Goldin - Eli Glickman
Checking Stuyvesant (NY)
Found 1 teams
Scraping Stuyvesant Jeremy Lee - Justin Sword
Checking Summit (NJ)
Found 2 teams
Scraping Summit Jeremiah Cohn - Anthony Yu
Scraping Summit Eray Sabuncu - Alex Supran
Checking Summit SYS (MA)
Found 1 teams
Scraping Summit SYS Rebekah Sun - Selina Tang
Checking Syosset (NY)
Found 0 teams
Checking Thatsnotcashmoney (TX)
Found 0 teams
Checking The Memorial Novices (TX)
Found 1 teams
Scraping The Memorial Novices Tommy Yu - Sebastian Cho
Checking The SCOTTS (FL)
Found 5 teams
Scraping The SCOTTS Evan Evan - Burkeen Burkeen
Scraping The SCOTTS Alyan Zafar - Isaac Ratzan
Scraping The SCOTTS Ethan Roytman - Sharvaa Selvan
Scraping The SCOTTS Jin Kwon - Christina Zhang
Scraping The SCOTTS Ethan Roytman - Mason the chad Cheng
Checking The White House (CA)
Found 1 teams
Scraping The White House Adam Mimou - no one
Checking TheBirdsWorkForTheBourgeoisie (TX)
Found 0 teams
Checking Thomas Woot

Scraping Anderson Ilan Benavi - Zach Domsch
Scraping Anderson Will Erard - Dylan Canyon
Scraping Anderson Ben Coleman - Piper Domsch
Checking Ardrey Kell (NC)
Found 1 teams
Scraping Ardrey Kell Emma Rogers - Sreyas Sureneni
Checking Aurorae Young Academy (WA)
Found 4 teams
Scraping Aurorae Young Academy Kailani Ye - Elizabeth Chen
Scraping Aurorae Young Academy Mingmei Sun - Cynthia Lu
Scraping Aurorae Young Academy Judy Wang - Maggie Yan
Scraping Aurorae Young Academy Evida Wei - Annabelle Chen
Checking Basis Independent Silicon Valley (CA)
Found 2 teams
Scraping Basis Independent Silicon Valley Stefan Boone - Daniel Chen
Scraping Basis Independent Silicon Valley Vinay Vellore - Ayush Agarwal
Checking Basis Peoria (AZ)
Found 3 teams
Scraping Basis Peoria Yash Lal - Samuel Parau
Scraping Basis Peoria Mateen Sekandari - Pranav Saravanan
Scraping Basis Peoria Samuel Parau - Haris Kurspahic
Checking Basis Phoenix (AZ)
Found 1 teams
Scraping Basis Phoenix Siddu Jandhyala - Andrew Yuan
Chec

Scraping Dulles Aidan Zhu - Jadon Zheng
Checking E D (PA)
Found 1 teams
Scraping E D Rajat Reddy - Rajat Reddee
Checking Eagan (MN)
Found 1 teams
Scraping Eagan Abhilash Ari - Sriram Nutulapati
Checking Eagle (ID)
Found 1 teams
Scraping Eagle Brandon Friend - Jacob Araki
Checking East Lake (FL)
Found 0 teams
Checking East Ridge (MN)
Found 1 teams
Scraping East Ridge Srujan Miryala - Lan Do
Checking Edgemont (NY)
Found 0 teams
Checking Edina (MN)
Found 8 teams
Scraping Edina Arjun Maheshwari - Ryan Zhu
Scraping Edina Evan Jiang - Isaac Wu
Scraping Edina Arya Mirza - Ryan Jiang
Scraping Edina Jay Shapiro - Ashwin Maheshwari
Scraping Edina Abbi Shilts - Ziyan Ishani
Scraping Edina Isaac Smalley - Evan TessmerTuck
Scraping Edina Jay Shapiro - Ryan Jiang
Scraping Edina Evan Jiang - Ryan Jiang
Checking Elkins (TX)
Found 5 teams
Scraping Elkins Emad Khalid - Danny Vilson
Scraping Elkins Femi Aderinto - Armaan Idrees
Scraping Elkins Rithvik Edupuganti - Naman Janga
Scraping Elkins Tharun Nayar

Scraping Marist School Lianna Homrich - Elly Kang
Scraping Marist School Brayden Miller - Cole Miller
Scraping Marist School Ethan Ekpenyong - Melissa Valencia
Scraping Marist School Maggie Nahas - Melissa Valencia
Scraping Marist School Ber Erwin - Ian Chung
Scraping Marist School Elly Kang - Sydney Vance
Scraping Marist School Ian Chung - Elly Kang
Scraping Marist School Claire Lauterbach - Sydney Vance
Checking Masterman (PA)
Found 1 teams
Scraping Masterman Noah Eggerts - Sammie Keenan
Checking McDowell (PA)
